In [1]:
import dolfin as dl
import hippylib as hl
import numpy as np

from spin.core import problem
from spin.hippylib import hessian, laplace, misfit, optimization, prior

In [2]:
mesh = dl.IntervalMesh(100, -1, 1)
problem_settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time",
    inference_type="drift_only",
    drift=("-x[0]",),
    log_squared_diffusion=("std::log(1.0)",),
)
problem_builder = problem.SPINProblemBuilder(problem_settings)
spin_problem = problem_builder.build()

In [3]:
prior_settings = prior.PriorSettings(
    function_space=spin_problem.function_space_parameters,
    mean=("-0.5*x[0]",),
    variance=("0.1",),
    correlation_length=("0.1",),
)
prior_builder = prior.BilaplacianVectorPriorBuilder(prior_settings)
spin_prior = prior_builder.build()
prior_variance = spin_prior.compute_variance_with_boundaries(
    method="Randomized", num_eigenvalues_randomized=50
)

In [4]:
observation_points = np.array((-0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8))
exact_solution = np.array(
    (
        0.21648217,
        0.42039515,
        0.58953849,
        0.70196354,
        0.74143722,
        0.70196354,
        0.58953849,
        0.42039515,
        0.21648217,
    )
)
perturbed_solution = exact_solution + np.random.normal(0, 0.01, exact_solution.shape)
noise_variance = 0.01 * np.ones(exact_solution.shape)

misfit_settings = misfit.MisfitSettings(
    function_space=spin_problem.function_space_variables,
    observation_points=observation_points,
    observation_values=perturbed_solution,
    noise_variance=noise_variance,
)
misfit_builder = misfit.MisfitBuilder(misfit_settings)
spin_misfit = misfit_builder.build()

In [5]:
inference_model = hl.Model(
    spin_problem.hippylib_variational_problem, spin_prior.hippylib_prior, spin_misfit
)

In [6]:
optimization_settings = optimization.SolverSettings(
    relative_tolerance=1e-3, absolute_tolerance=1e-3, verbose=False
)
initial_guess = spin_prior.mean_array
newton_solver = optimization.NewtonCGSolver(optimization_settings, inference_model)
solver_solution = newton_solver.solve(initial_guess)

In [7]:
hessian_settings = hessian.LowRankHessianSettings(
    num_eigenvalues=15,
    num_oversampling=5,
    inference_model=inference_model,
    evaluation_point=[
        solver_solution.forward_solution,
        solver_solution.optimal_parameter,
        solver_solution.adjoint_solution,
    ],
)
eigenvalues, eigenvectors = hessian.compute_low_rank_hessian(hessian_settings)

In [ ]:
laplace_approximation_settings = laplace.LowRankLaplaceApproximationSettings(
    inference_model=inference_model,
    mean=solver_solution.optimal_parameter,
    low_rank_hessian_eigenvalues=eigenvalues,
    low_rank_hessian_eigenvectors=eigenvectors,
)
laplace_approximation = laplace.LowRankLaplaceApproximation(laplace_approximation_settings)
posterior_variance = laplace_approximation.compute_pointwise_variance(
    method="Randomized", num_eigenvalues_randomized=50
)